In [0]:

from  nltk.tokenize  import RegexpTokenizer
from nltk.corpus import stopwords
from stop_words import get_stop_words
from nltk.stem.snowball import SnowballStemmer
from gensim import corpora, models
import gensim
import nltk
import  pandas  as  pd
import  gzip

In [16]:

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

pos = gc.open('nake_pos.xlsx').sheet1

# get_all_values gives a list of rows.
rows = pos.get_all_values()


# Convert to a DataFrame and render.
import pandas as pd
df_pos=pd.DataFrame.from_records(rows)

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

neg = gc.open('nake_neg.xlsx').sheet1

# get_all_values gives a list of rows.
rows = neg.get_all_values()


# Convert to a DataFrame and render.
import pandas as pd
df_neg=pd.DataFrame.from_records(rows)

In [54]:
df_neg.head()


,rating,review
0,rating,reviews
1,1,I bought this a couple months ago and have tri...
2,1,These eyeshadows have ok payoff they dont last...
3,1,I bought this after seeing all the hype that c...
4,1,These eyeshadows have ok payoff they dont last...


In [12]:
df_pos.columns = ['rating', 'review']
df_neg.columns = ['rating', 'review']
df_pos=df_pos.drop([0])
df_neg=df_neg.drop([0])
df_pos.head()

,rating,review
1,5,I'm completely in love with this pallete!! Gre...
2,5,I bought this pallet after having a mini lesso...
3,5,I don't think there is one color in this palet...
4,5,So worth the money. Gorgeous colors. Applies l...
5,5,I got this palette as a gift because I have be...


In [61]:

df_neg.head()

,rating,review
1,1,I bought this a couple months ago and have tri...
2,1,These eyeshadows have ok payoff they dont last...
3,1,I bought this after seeing all the hype that c...
4,1,These eyeshadows have ok payoff they dont last...
5,1,This is the worst palette I've ever use in my ...


In [37]:
type(df_neg)

pandas.core.frame.DataFrame

In [14]:
tokenizer = RegexpTokenizer(r'\w+')
doc_1 = df_pos.review[1]

# Using one of our docs as an example# Using 
tokens = tokenizer.tokenize(doc_1.lower())

print('{} characters in string vs {} words in a list'.format(len(doc_1),                                                             len(tokens)))
print(tokens[:10])

236 characters in string vs 37 words in a list
['i', 'm', 'completely', 'in', 'love', 'with', 'this', 'pallete', 'great', 'natural']


In [19]:
nltk_stpwd  = stopwords.words('english')
stop_words_stpwd = get_stop_words('en')
merged_stopwords = list(set(nltk_stpwd + stop_words_stpwd))

print(len(set(merged_stopwords)))
print(merged_stopwords[:10])

211
['for', 'further', 'could', 'that', 'same', "i'd", 'your', "we'd", 'yourselves', 'you']


In [20]:
stopped_tokens = [token for token in tokens if not token in merged_stopwords]
print(stopped_tokens[:10])

['completely', 'love', 'pallete', 'great', 'natural', 'looks', 'favorite', 'combination', 'snakebite', 'half']


In [21]:
sb_stemmer = SnowballStemmer('english')
stemmed_tokens  = [sb_stemmer.stem(token) for token in stopped_tokens]
print(stemmed_tokens)

['complet', 'love', 'pallet', 'great', 'natur', 'look', 'favorit', 'combin', 'snakebit', 'half', 'bake', 'copper', 'give', 'natur', 'adriana', 'lima', 'smokey', 'eye', 'look', 'definit', 'recommend', 'pallet', 'love']


In [22]:
%%time

num_reviews = df_pos.shape[0]

doc_set = [df_pos.review[i] for i in range(1, num_reviews)]

texts = []

for doc in doc_set:
    # putting our three steps together
    tokens = tokenizer.tokenize(doc.lower())
    stopped_tokens = [token for token in tokens if not token in merged_stopwords]
    stemmed_tokens = [sb_stemmer.stem(token) for token in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)


CPU times: user 6.46 s, sys: 15 ms, total: 6.47 s
Wall time: 6.48 s


In [23]:
%%time

num_reviews = df_neg.shape[0]

doc_set = [df_neg.review[i] for i in range(1, num_reviews)]

texts_neg = []

for doc in doc_set:
    # putting our three steps together
    tokens = tokenizer.tokenize(doc.lower())
    stopped_tokens = [token for token in tokens if not token in merged_stopwords]
    stemmed_tokens = [sb_stemmer.stem(token) for token in stopped_tokens]
    
    # add tokens to list
    texts_neg.append(stemmed_tokens)

CPU times: user 155 ms, sys: 0 ns, total: 155 ms
Wall time: 157 ms


In [82]:
texts[0]

['complet',
 'love',
 'pallet',
 'great',
 'natur',
 'look',
 'favorit',
 'combin',
 'snakebit',
 'half',
 'bake',
 'copper',
 'give',
 'natur',
 'adriana',
 'lima',
 'smokey',
 'eye',
 'look',
 'definit',
 'recommend',
 'pallet',
 'love']

In [24]:
textsneg_dict = corpora.Dictionary(texts_neg)
textsneg_dict.save('negreview.dict') 
corpus = [textsneg_dict.doc2bow(text) for text in texts_neg]
gensim.corpora.MmCorpus.serialize('review_neg.mm', corpus)
lda_model = gensim.models.LdaModel(corpuspos,alpha='auto', num_topics=5,id2word=texts_dict, passes=20)
lda_model.show_topics(num_topics=5,num_words=5)

Dictionary(1011 unique tokens: ['ago', 'bought', 'color', 'coupl', 'differ']...)


In [110]:
# Gensim's Dictionary encapsulates the mapping between normalized words and their integer ids.# Gensim 
texts_dict = corpora.Dictionary(texts)
texts_dict.save('review.dict') # lets save to disk for later use
# Examine each token’s unique id
print(texts_dict)

Dictionary(1011 unique tokens: ['ago', 'bought', 'color', 'coupl', 'differ']...)


In [98]:
import operator
print("IDs 1 through 10: {}".format(sorted(textsneg_dict.token2id.items(), key=operator.itemgetter(1), reverse = False)[:10]))

IDs 1 through 10: [('ago', 0), ('bought', 1), ('color', 2), ('coupl', 3), ('differ', 4), ('eye', 5), ('fallout', 6), ('insan', 7), ('look', 8), ('make', 9)]


In [85]:
print(sorted(texts_dict.token2id.items(), key=operator.itemgetter(1), reverse = False)[:10])

[('adriana', 0), ('bake', 1), ('combin', 2), ('complet', 3), ('copper', 4), ('definit', 5), ('eye', 6), ('favorit', 7), ('give', 8), ('great', 9)]


In [25]:
corpus = [textsneg_dict.doc2bow(text) for text in texts_neg]
len(corpus)

386

In [26]:
corpus

[[(0, 1),
  (1, 1),
  (2, 3),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1)],
 [(20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1)],
 [(1, 1),
  (19, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 2),
  (47, 1),
  (48, 1),
  (49, 1)],
 [(20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1)],
 [(0, 1),
  (1, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1)],
 [(2, 1),
  (5, 1),
  (8, 1),
  (43, 1),
  (69, 1),

In [115]:
corpuspos = [texts_dict.doc2bow(text) for text in texts]
len(corpuspos)

17098

In [100]:
%%time 
# Matrix Market format https://radimrehurek.com/gensim/corpora/mmcorpus.html, why exactly? I don't know
gensim.corpora.MmCorpus.serialize('review_neg.mm', corpus)

CPU times: user 11 ms, sys: 5 ms, total: 16 ms
Wall time: 22.4 ms


In [116]:
%%time 
# Matrix Market format https://radimrehurek.com/gensim/corpora/mmcorpus.html, why exactly? I don't know
gensim.corpora.MmCorpus.serialize('reviewpos.mm', corpuspos)

CPU times: user 403 ms, sys: 35 ms, total: 438 ms
Wall time: 445 ms


In [117]:
%%time 
lda_model = gensim.models.LdaModel(corpuspos,alpha='auto', num_topics=5,id2word=texts_dict, passes=20)

CPU times: user 1min 23s, sys: 213 ms, total: 1min 23s
Wall time: 1min 23s


In [118]:
lda_model.show_topics(num_topics=5,num_words=5)

[(0,
  '0.142*"look" + 0.049*"shade" + 0.040*"eye" + 0.035*"creat" + 0.032*"matt"'),
 (1,
  '0.261*"nake" + 0.092*"urban" + 0.090*"decay" + 0.071*"2" + 0.064*"first"'),
 (2,
  '0.142*"skin" + 0.128*"tone" + 0.125*"best" + 0.089*"ever" + 0.053*"eye"'),
 (3,
  '0.076*"color" + 0.075*"palett" + 0.069*"love" + 0.032*"use" + 0.028*"great"'),
 (4, '0.042*"bought" + 0.035*"got" + 0.028*"get" + 0.026*"buy" + 0.023*"tri"')]

In [125]:
%%time 
lda_model = gensim.models.LdaModel(corpus,alpha='auto', num_topics=5,id2word=textsneg_dict, passes=20)

CPU times: user 4.62 s, sys: 1 ms, total: 4.62 s
Wall time: 4.62 s


In [126]:
lda_model.show_topics(num_topics=5,num_words=5)

[(0,
  '0.044*"color" + 0.029*"palett" + 0.022*"nake" + 0.017*"use" + 0.016*"pigment"'),
 (1,
  '0.039*"palett" + 0.036*"color" + 0.026*"look" + 0.018*"tri" + 0.016*"like"'),
 (2,
  '0.036*"pigment" + 0.027*"palett" + 0.024*"money" + 0.023*"color" + 0.022*"worth"'),
 (3,
  '0.041*"color" + 0.036*"palett" + 0.027*"like" + 0.021*"look" + 0.020*"get"'),
 (4,
  '0.066*"shade" + 0.043*"matt" + 0.031*"pigment" + 0.028*"primer" + 0.026*"even"')]